In [48]:
import spacy
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

In [18]:
kg_df = pd.read_csv("../cskg_star.tsv", sep="\t", error_bad_lines=False)

/tmp/ipykernel_2746282/3848696862.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  kg_df = pd.read_csv("../cskg_star.tsv", sep="\t", error_bad_lines=False)
b'Skipping line 5997119: expected 10 fields, saw 11\nSkipping line 6016458: expected 10 fields, saw 11\nSkipping line 6017377: expected 10 fields, saw 11\n'
b'Skipping line 6049906: expected 10 fields, saw 11\nSkipping line 6067719: expected 10 fields, saw 11\nSkipping line 6069988: expected 10 fields, saw 11\nSkipping line 6079377: expected 10 fields, saw 11\nSkipping line 6092900: expected 10 fields, saw 11\n'
b'Skipping line 6101744: expected 10 fields, saw 11\nSkipping line 6101793: expected 10 fields, saw 11\nSkipping line 6131769: expected 10 fields, saw 11\n'
b'Skipping line 6171924: expected 10 fields, saw 11\nSkipping line 6215673: expected 10 fields, saw 11\nSkipping line 6223428: expected 10 fields, saw 11\nSkipping line 6225

In [19]:
kg_df.head(3)

,id,node1,relation,node2,node1;label,node2;label,relation;label,relation;dimension,source,sentence
0,/c/en/0-/r/DefinedAs-/c/en/empty_set-0000,/c/en/0,/r/DefinedAs,/c/en/empty_set,0,empty set,defined as,NaN,CN,[[0]] is the [[empty set]].
1,/c/en/0-/r/DefinedAs-/c/en/first_limit_ordinal...,/c/en/0,/r/DefinedAs,/c/en/first_limit_ordinal,0,first limit ordinal,defined as,NaN,CN,[[0]] is the [[first limit ordinal]].
2,/c/en/0-/r/DefinedAs-/c/en/number_zero-0000,/c/en/0,/r/DefinedAs,/c/en/number_zero,0,number zero,defined as,NaN,CN,[[0]] is the [[number zero]].


In [ ]:
all_kg_sentences = []
print(kg_df.shape)
for _, row in tqdm(kg_df.iterrows()):
    if str(row['sentence']) != 'nan':
        all_kg_sentences.append(row['sentence'])
    else:
        all_kg_sentences.append(f"{row['node1;label']} {row['relation;label']} {row['node2;label']}")

In [53]:
all_kg_sentences[200:300]

['0r has context nonstandard',
 '0s form of 0',
 '0s related to 0',
 '0th derived from 0',
 '0tlps form of 0tlp',
 '[[1]] can [[equal 1]]',
 '1 has context internet',
 '1 has context internet slang',
 '1 has context leet',
 '1 has context sarcastic',
 '1 has context slang',
 '[[1]] is [[one]]',
 '[["1"]] is [[pronounced one]]',
 '[[1]] is a kind of [[Abstaction]].',
 '[[1]] is [[a higher number than 0]]',
 '[[1]] is [[a number]]',
 '[[{1}]] is a type of [[set containing one element]].',
 '1 related to draft',
 '1 related to like',
 '1 related to number',
 '1 related to one',
 '1 synonym one',
 '1,000 megawatt plutonium reactor is a tangible thing',
 '1,000 meter form of 1000 metre',
 '1,000 metre form of 1000 metre',
 '1,1,1 trichloro 2,2 bis 4 chlorophenyl ethane related to dichlorodiphenyltrichloroethane',
 '1,1,2 trichloro 1,2,2 trifluoroethane is a chemical compound',
 '1,1,2 trichloro 1,2,2 trifluoroethane synonym 1,1,2 trichloro 1,2,2 trifluoroethane',
 '1,1,2 trichloro 1,2,2 tri

In [56]:

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

sentences = ["This is an example sentence", "Each sentence is converted"] * 100

embeddings = model.encode(sentences)


In [57]:
embeddings.shape

(200, 768)

In [4]:
with open('train.source', 'r', encoding='utf-8-sig') as f:
    train_source = f.read().splitlines()

In [5]:
unique_sentences = list(set(train_source))
len(unique_sentences)

2368

In [12]:
beliefs = []
arguments = []
for sent in unique_sentences:
    all_seps = [m.start() for m in re.finditer('</s>', sent)]
    beliefs.append(sent[7:all_seps[0]])
    arguments.append(sent[all_seps[0]+14:all_seps[1]])